In [ ]:
#| default_exp routes.group

In [ ]:
#| exporti
import httpx

import domolibrary.client.get_data as gd
import domolibrary.client.ResponseGetData as rgd
import domolibrary.client.DomoAuth as dmda
import domolibrary.client.DomoError as de

In [ ]:
# | export
def generate_body_create_group(group_name: str,
                               group_type: str = 'open',
                               description: str = '') -> dict:
    """ Generates the body to create group for content_v2_group API"""
    body = {"name": group_name, 
            "type": group_type,
            "description": description}

    return body

#### sample implementation of generate_body_create_group

In [ ]:
generate_body_create_group(
    group_name='test_group_name', 
    group_type='open', 
    description='from jupyter')


{'name': 'test_group_name', 'type': 'open', 'description': 'from jupyter'}

In [ ]:
#| export
class CreateGroup_Error(de.DomoError):
    def __init__(self, status, message, domo_instance, function_name = "create_group"):
        super().__init__(function_name = function_name, status = status, message = message , domo_instance = domo_instance)

async def create_group(auth: dmda.DomoAuth,
                       group_name: str,
                       group_type: str = 'open',
                       description: str = '',
                       debug_api: bool = False,
                       session: httpx.AsyncClient = None
                       ) -> rgd.ResponseGetData:
    # body : {"name": "GROUP_NAME", "type": "open", "description": ""}

    body = generate_body_create_group(
        group_name=group_name, group_type=group_type, description=description)

    url = f'https://{auth.domo_instance}.domo.com/api/content/v2/groups/'

    res= await gd.get_data(
        auth=auth,
        url=url,
        method='POST',
        body=body,
        debug_api=debug_api,
        session = session
    )

    if not res.is_success:
        raise CreateGroup_Error(
            status = res.status, 
            message = res.response,
            domo_instance = auth.domo_instance, 
            function_name="create_group")

    return res


#### Sample implementation of create_group

In [ ]:
import os

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-dojo", 
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"]
)
try:
    res = await create_group(auth=token_auth, group_name='Test Group 2', debug_api=False)
    res.response

except CreateGroup_Error as e:
    print(e)

create_group: Status 400 - Bad Request at domo-dojo


In [ ]:
# #| export
# async def get_all_groups(auth: dmda.DomoAuth,
#                          log_results: bool = False,
#                          debug: bool = False,
#                          session: httpx.AsyncClient = None) -> rgd:
#     if debug:
#         print(auth)

#     url = f'https://{auth.domo_instance}.domo.com/api/content/v2/groups/grouplist'

#     if debug:
#         print(auth, url)

#     if log_results:
#         print(f'Getting groups from - {url}')

#     res = await gd(url=url, method='GET', auth=auth, session=session)

#     return res

In [ ]:
# #| export
# async def search_groups_by_name(auth: DomoFullAuth,
#                                 search_name: str,
#                                 debug: bool = False, log_results: bool = False) -> rgd:
#     url = f'https://{auth.domo_instance}.domo.com/api/content/v2/groups/grouplist?ascending=true&search={search_name}&sort=name '

#     res = await gd(
#         auth=auth,
#         url=url,
#         method='GET',
#         debug=debug,
#         log_results=log_results
#     )
#     return res

In [ ]:
#| export
def generate_body_update_group_membership(group_id: str,
                                          add_user_arr: list[str] = None,
                                          remove_user_arr: list[str] = None,
                                          add_owner_user_arr: list[str] = None,
                                          remove_owner_user_arr: list[str] = None) -> list[dict]:
    body = {"groupId": int(group_id)}
    if add_owner_user_arr:
        body.update({"addOwners": [{"type": "USER", "id": str(
            userId)} for userId in add_owner_user_arr]})

    if remove_owner_user_arr:
        body.update({"removeOwners": [{"type": "USER", "id": str(
            userId)} for userId in remove_owner_user_arr]})

    if remove_user_arr:
        body.update({"removeMembers": [
                    {"type": "USER", "id": str(userId)} for userId in remove_user_arr]})
    if add_user_arr:
        body.update(
            {"addMembers": [{"type": "USER", "id": str(userId)} for userId in add_user_arr]})

    return [body]

In [ ]:
# #| export
# async def update_group_membership(auth: DomoFullAuth,
#                                   body: dict,
#                                   log_results: bool = False, debug: bool = False) -> ResponseGetData:
#     # body = [{
#     #     "groupId":"GROUP_ID",
#     #     "removeMembers": [{"type":"USER","id":"USER_ID"}],
#     #     "addMembers"   : [{"type":"USER","id":"USER_ID"}]
#     # }]

#     url = f'https://{auth.domo_instance}.domo.com/api/content/v2/groups/access'

#     if debug:
#         print(url, body)

#     res = await gd(
#         auth=auth,
#         url=url,
#         method='PUT',
#         body=body,
#         debug=debug
#     )

#     return res

In [ ]:
# #| export
# async def get_group_by_id(auth:DomoFullAuth, group_id:id, debug:bool = False) -> ResponseGetData:
#     url = f'https://{auth.domo_instance}.domo.com/api/content/v2/groups/{group_id}'

#     if debug:
#         print(url)

#     res = await gd(
#         auth=auth,
#         url=url,
#         method='GET',
#         debug=debug
#     )

#     return res

In [ ]:
#| hide
import nbdev
nbdev.nbdev_export()
